In [149]:
import os
import glob
import pandas as pd
import keras
import tensorflow as tf
import pickle

In [150]:
# load csv
inter_raw = pd.read_csv('../../data/RAW_interactions.csv')
recipes_raw = pd.read_csv('../../data/RAW_recipes.csv')
recipies_pp = pd.read_csv('../../data/PP_recipes.csv')



In [151]:
with open('../../data/ingr_map.pkl', 'rb') as f:
    ingr_map = pickle.load(f)

In [152]:
ingr_map

,raw_ingr,raw_words,processed,len_proc,replaced,count,id
0,"medium heads bibb or red leaf lettuce, washed,...",13,"medium heads bibb or red leaf lettuce, washed,...",73,lettuce,4507,4308
1,mixed baby lettuces and spring greens,6,mixed baby lettuces and spring green,36,lettuce,4507,4308
2,romaine lettuce leaf,3,romaine lettuce leaf,20,lettuce,4507,4308
3,iceberg lettuce leaf,3,iceberg lettuce leaf,20,lettuce,4507,4308
4,red romaine lettuce,3,red romaine lettuce,19,lettuce,4507,4308
...,...,...,...,...,...,...,...
11654,soybeans,1,soybean,7,soybean,31,6702
11655,goose,1,goose,5,goose,8,3318
11656,ajwain,1,ajwain,6,ajwain,13,47
11657,brinjals,1,brinjal,7,brinjal,2,750


In [153]:
recipies_pp.head()

,id,i,name_tokens,ingredient_tokens,steps_tokens,techniques,calorie_level,ingredient_ids
0,424415,23,"[40480, 37229, 2911, 1019, 249, 6878, 6878, 28...","[[2911, 1019, 249, 6878], [1353], [6953], [153...","[40480, 40482, 21662, 481, 6878, 500, 246, 161...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[389, 7655, 6270, 1527, 3406]"
1,146223,96900,"[40480, 18376, 7056, 246, 1531, 2032, 40481]","[[17918], [25916], [2507, 6444], [8467, 1179],...","[40480, 40482, 729, 2525, 10906, 485, 43, 8393...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[2683, 4969, 800, 5298, 840, 2499, 6632, 7022,..."
2,312329,120056,"[40480, 21044, 16954, 8294, 556, 10837, 40481]","[[5867, 24176], [1353], [6953], [1301, 11332],...","[40480, 40482, 8240, 481, 24176, 296, 1353, 66...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1,"[1257, 7655, 6270, 590, 5024, 1119, 4883, 6696..."
3,74301,168258,"[40480, 10025, 31156, 40481]","[[1270, 1645, 28447], [21601], [27952, 29471, ...","[40480, 40482, 5539, 21601, 1073, 903, 2324, 4...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0,"[7940, 3609, 7060, 6265, 1170, 6654, 5003, 3561]"
4,76272,109030,"[40480, 17841, 252, 782, 2373, 1641, 2373, 252...","[[1430, 11434], [1430, 17027], [1615, 23, 695,...","[40480, 40482, 14046, 1430, 11434, 488, 17027,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",0,"[3484, 6324, 7594, 243]"


In [141]:
recipies_pp.describe()


,id,i,calorie_level
count,178265.000000,178265.000000,178265.000000
mean,213461.803007,89132.000000,0.863192
std,138266.712754,51460.817206,0.791486
min,38.000000,0.000000,0.000000
25%,94576.000000,44566.000000,0.000000
50%,196312.000000,89132.000000,1.000000
75%,320562.000000,133698.000000,2.000000
max,537716.000000,178264.000000,2.000000


In [142]:
recipies_pp['recipe_id'] = recipies_pp.pop('id')

In [143]:

inter_raw.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [144]:
inter_raw.describe()

,user_id,recipe_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06
mean,1.384291e+08,1.608972e+05,4.411016e+00
std,5.014269e+08,1.303987e+05,1.264752e+00
min,1.533000e+03,3.800000e+01,0.000000e+00
25%,1.354700e+05,5.425700e+04,4.000000e+00
50%,3.309370e+05,1.205470e+05,5.000000e+00
75%,8.045500e+05,2.438520e+05,5.000000e+00
max,2.002373e+09,5.377160e+05,5.000000e+00


In [145]:
recipes_raw.describe()


,id,minutes,contributor_id,n_steps,n_ingredients
count,231637.000000,2.316370e+05,2.316370e+05,231637.000000,231637.000000
mean,222014.708984,9.398546e+03,5.534885e+06,9.765499,9.051153
std,141206.635626,4.461963e+06,9.979141e+07,5.995128,3.734796
min,38.000000,0.000000e+00,2.700000e+01,0.000000,1.000000
25%,99944.000000,2.000000e+01,5.690500e+04,6.000000,6.000000
50%,207249.000000,4.000000e+01,1.736140e+05,9.000000,9.000000
75%,333816.000000,6.500000e+01,3.982750e+05,12.000000,11.000000
max,537716.000000,2.147484e+09,2.002290e+09,145.000000,43.000000


In [146]:
recipes_raw['recipe_id'] = recipes_raw.pop('id')

In [147]:
omni_raw = pd.merge(recipes_raw,inter_raw,on='recipe_id')

In [148]:
omni_raw.describe()

,minutes,contributor_id,n_steps,n_ingredients,recipe_id,user_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06,1.132367e+06,1.132367e+06,1.132367e+06,1.132367e+06
mean,3.614684e+04,2.655967e+06,9.627970e+00,8.952028e+00,1.608972e+05,1.384291e+08,4.411016e+00
std,8.796494e+06,6.774670e+07,5.819689e+00,3.689908e+00,1.303987e+05,5.014269e+08,1.264752e+00
min,0.000000e+00,2.700000e+01,0.000000e+00,1.000000e+00,3.800000e+01,1.533000e+03,0.000000e+00
25%,2.000000e+01,3.744900e+04,6.000000e+00,6.000000e+00,5.425700e+04,1.354700e+05,4.000000e+00
50%,4.000000e+01,8.983100e+04,8.000000e+00,9.000000e+00,1.205470e+05,3.309370e+05,5.000000e+00
75%,7.000000e+01,2.427290e+05,1.200000e+01,1.100000e+01,2.438520e+05,8.045500e+05,5.000000e+00
max,2.147484e+09,2.002290e+09,1.450000e+02,4.300000e+01,5.377160e+05,2.002373e+09,5.000000e+00


In [154]:
omni_raw['recipe_features'] = omni_raw[['minutes','n_steps','n_ingredients']].values.tolist()
omni_raw = omni_raw.drop(['minutes','n_steps','n_ingredients', 'name', 'contributor_id', 'submitted', 'tags', 'nutrition', 'steps', 'description', 'review', 'ingredients'], axis=1)
omni_raw.head()

,recipe_id,user_id,date,rating,recipe_features
0,137739,4470,2006-02-18,5,"[55, 11, 7]"
1,137739,593927,2010-08-21,5,"[55, 11, 7]"
2,137739,178427,2011-12-05,5,"[55, 11, 7]"
3,31490,28603,2002-08-19,0,"[30, 9, 6]"
4,31490,346277,2006-08-27,5,"[30, 9, 6]"


In [155]:
omni_raw.describe()

,recipe_id,user_id,rating
count,1.132367e+06,1.132367e+06,1.132367e+06
mean,1.608972e+05,1.384291e+08,4.411016e+00
std,1.303987e+05,5.014269e+08,1.264752e+00
min,3.800000e+01,1.533000e+03,0.000000e+00
25%,5.425700e+04,1.354700e+05,4.000000e+00
50%,1.205470e+05,3.309370e+05,5.000000e+00
75%,2.438520e+05,8.045500e+05,5.000000e+00
max,5.377160e+05,2.002373e+09,5.000000e+00


In [156]:
omni_raw = pd.merge(omni_raw,recipies_pp.drop(['i','name_tokens','steps_tokens','techniques','calorie_level', 'ingredient_tokens',],axis = 1),on='recipe_id')
omni_raw.head()

KeyError: 'recipe_id'

In [ ]:
omni_raw.describe()

In [ ]:
omni_raw = omni_raw[omni_raw.rating >= 4].drop(['rating'], axis = 1)
omni_raw.head()

,recipe_id,user_id,date,recipe_features,ingredient_ids
0,137739,4470,2006-02-18,"[55, 11, 7]","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
1,137739,593927,2010-08-21,"[55, 11, 7]","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
2,137739,178427,2011-12-05,"[55, 11, 7]","[7933, 4694, 4795, 3723, 840, 5006, 6270]"
4,31490,346277,2006-08-27,"[30, 9, 6]","[5481, 6324, 2499, 4717, 6276, 1170]"
5,31490,319943,2009-07-18,"[30, 9, 6]","[5481, 6324, 2499, 4717, 6276, 1170]"


In [ ]:
omni_raw = omni_raw.sort_values(['user_id','date'])
omni_raw.head()

,recipe_id,user_id,date,recipe_features,ingredient_ids
932353,17338,1533,2002-02-19,"[50, 10, 13]","[2925, 3484, 5010, 3203, 6270, 382, 5106, 5319..."
237828,24375,1533,2002-04-23,"[15, 6, 9]","[422, 6273, 7258, 5365, 893, 2131, 2783, 4253,..."
615125,10721,1533,2002-05-02,"[40, 3, 4]","[7705, 6906, 2853, 5502]"
633277,23891,1533,2002-05-06,"[25, 7, 7]","[7753, 5495, 2499, 2404, 5180, 4785, 4958]"
317699,22736,1533,2002-05-20,"[1455, 13, 6]","[1252, 519, 4545, 6696, 800, 5006]"


In [ ]:
omni_raw['n_inter'] = omni_raw.value_counts('user_id')
omni_raw = omni_raw[omni_raw['n_inter'] >= 10]
omni_raw.head()

,recipe_id,user_id,date,recipe_features,ingredient_ids,n_inter
678366,25548,1533,2002-07-01,"[75, 11, 13]","[6220, 5010, 7987, 3440, 3184, 7946, 2200, 339...",386.0
147109,14807,1533,2002-08-19,"[19, 7, 7]","[5373, 912, 840, 800, 1591, 4958, 1511]",113.0
394810,8566,1535,2005-05-26,"[35, 6, 12]","[4998, 6810, 869, 4987, 800, 2499, 7887, 332, ...",18.0
22995,140366,1535,2007-06-05,"[20, 9, 12]","[1388, 1910, 2499, 6270, 6906, 2683, 1584, 755...",29.0
612835,17614,1535,2007-07-12,"[70, 6, 10]","[2683, 800, 332, 6270, 840, 2499, 4717, 5068, ...",13.0


In [ ]:
omni_raw.describe()

,recipe_id,user_id,n_inter
count,7698.000000,7.698000e+03,7698.000000
mean,150999.368277,9.427179e+07,66.011432
std,125193.837826,4.177131e+08,186.735535
min,62.000000,1.533000e+03,10.000000
25%,51210.000000,1.269662e+05,14.000000
50%,110177.500000,3.020940e+05,22.000000
75%,225198.500000,6.583210e+05,46.000000
max,533649.000000,2.002366e+09,4494.000000


In [ ]:
omni_test = omni_raw
counts = omni_test.value_counts('user_id')
omni_test = omni_test.loc[omni_test['user_id'].isin(counts.index[counts >= 6])]



In [ ]:
omni_test.describe()

,recipe_id,user_id,n_inter
count,1861.000000,1.861000e+03,1861.000000
mean,209398.573885,6.690454e+06,65.177861
std,134995.501932,1.134239e+08,187.535109
min,445.000000,1.535000e+03,10.000000
25%,95950.000000,6.550200e+04,14.000000
50%,191389.000000,1.694300e+05,22.000000
75%,307439.000000,3.697150e+05,45.000000
max,533649.000000,2.000498e+09,4494.000000


In [ ]:
omni_test.value_counts('user_id')

user_id
424680    53
37449     46
383346    34
128473    28
199848    27
          ..
37636     10
29782     10
22015     10
17803     10
130819    10
Length: 70, dtype: int64